# Иcточники данных

- Внешние источники
  - b1: `b1_analytics.pkl`
  - kamaflow: `kamaflow_researches.pkl`
- Внутренние источники
  - Отраслевой хаб: данные выложены в sberdisk

## Формат данных 

Результат представляет собой массив словарей, где каждый словарь содержит информацию об одном аналитическом материале. Каждый словарь имеет следующие ключи:

### Ключи словаря:
1. **`link`** (`str`):
   - Ссылка на страницу с аналитическим материалом. Может быть как полная, так и относительная
   - Пример: `"/news/12345"`.

2. **`date`** (`str`):
   - Дата публикации материала.
   - Пример: `"2023-10-15"`.

3. **`name`** (`str`):
   - Название материала на русском языке.
   - Пример: `"Анализ рынка ИТ в 2023 году"`.

4. **`name_en`** (`str`):
   - Название материала на английском языке, извлеченное из ссылки.
   - Пример: `"it-market-analysis-2023"`.

5. **`tags`** (`list` of `str`):
   - Список тегов, связанных с материалом.
   - Пример: `["ИТ", "аналитика", "2023"]`.

6. **`pdf_links`** (`list` of `str`):
   - Список ссылок на PDF-файлы, связанные с материалом.
   - Пример: `["https://b1.ru/local/assets/surveys/it-market-analysis-2023.pdf"]`.

7. **`text`** (`str`):
   - Основной текст материала из статьи на сайте (в pdf больше информации), очищенный от лишних элементов (контакты, реклама, скрипты и т.д.).
   - Пример: `"В 2023 году рынок ИТ показал рост на 14%..."`.

8. **`pdfs`** (`list` of `dict`):
   - Список словарей, содержащих информацию о скачанных PDF-файлах. Каждый словарь содержит:
     - **`name`** (`str`): Название PDF-файла.
       - Пример: `"it-market-analysis-2023.pdf"`.
     - **`content`** (`bytes`): Бинарное содержимое PDF-файла.

---

### Пример элемента списка `analytics`:
```json
{
    "link": "/news/12345",
    "date": "2023-10-15",
    "name": "Анализ рынка ИТ в 2023 году",
    "name_en": "it-market-analysis-2023",
    "tags": ["ИТ", "аналитика", "2023"],
    "pdf_links": [
        "https://b1.ru/local/assets/surveys/it-market-analysis-2023.pdf"
    ],
    "text": "В 2023 году рынок ИТ показал рост на 14%...",
    "pdfs": [
        {
            "name": "it-market-analysis-2023.pdf",
            "content": "binary_pdf_content_here"
        }
    ]
}
```

In [2]:
import pickle

with open('b1_analytics.pkl', 'rb') as f:
    b1_analytics = pickle.load(f)

with open('kamaflow_researches.pkl', 'rb') as f:
    kamaflow_researches = pickle.load(f)
    
docs = b1_analytics + kamaflow_researches

## Саммаризация

In [ ]:
# from langchain_openai import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyPDFLoader
from langchain.schema import Document
import tempfile


# llm = ChatOpenAI(
#     model_name="gpt-4o",
#     )

for doc in docs:
    name = doc['name']
    tags = doc['tags']
    additional_text = doc['text']
    
    for pdf in doc['pdfs']:
        pdf_name = pdf['name']
        print(pdf_name)
        pdf_content = pdf['content']
        try:
            with tempfile.NamedTemporaryFile() as temp_file:
                temp_file.write(pdf_content)
                temp_file_path = temp_file.name
                loader = PyPDFLoader(temp_file_path)
                pdf_data = loader.load()
        except Exception as e:
            print(f"Error processing PDF {pdf_name}: {e}")
            pdf_data = None
        if pdf_data:
            full_text = "\n".join([doc.page_content for doc in pdf_data])

            prompt = f"""
            **Название документа:**  {name}
            **Теги документа:** {", ".join(tags)} 
            **Краткое содержание документа с сайта (возможны html артефакты):**  {additional_text}
            **Документ:** {full_text}

            {f"""
            Перед тобой текст, полученный из PDF-документа. Название документа: {pdf_name}. Он может содержать пропуски, так как изображения и таблицы не были извлечены.

            **Твоя задача** – провести детальный отраслевой анализ на основе этого текста на русском языке. Если информация обрывочна, попытайся восстановить логический контекст или отметь, какие данные отсутствуют.

            **Формат анализа:**
            1. **Ключевые выводы** – выдели главных мыслей.
            2. **Тенденции в отрасли** – связаны ли идеи с актуальными событиями или трендами? краткосрочные и долгосрочные прогнозы.
            3. **Анализ рисков, неопределённости и возможностей** – есть ли критические моменты или перспективные направления?
            4. **Основные показатели** – ключевые цифры, тренды, прогнозы.
            5. **Рыночная динамика** – влияние факторов на отрасль.

            **Выведи аналитический отчёт в структурированном виде:**
            """}
            """

            # summarize_chain = load_summarize_chain(llm, chain_type="stuff")
            # summary = summarize_chain.invoke([Document(page_content=prompt)])
            pdf['content'] = 'b'
            pdf['summary'] = 'a'# summary['output_text']
        else:
            pdf['content'] = ''
            pdf['summary'] = ''

b1-dsight-venture-eurasia-2024-results-review.pdf
b1-overview-of-russian-companies-ifrs-s2-climate-related-reporting-2024.pdf
b1-consumer-products-sector-survey-november-2024.pdf
b1-mts-ai-deepfake-and-spoofing-threats-for-business-survey-2025.pdf
b1-survey-on-ifrs-17-implementation-in-insurance-sector-actuarial-calculations-2024.pdf
b1-ofs-industry-trends-survey-2024.pdf
b1-russian-forest-industry-2024-overview.pdf
b1-survey-on-ifrs-17-implementation-in-insurance-sector-2024.pdf
b1-urban-environment-in-megacities-survey.pdf
b1-agro-business-tendencies-in-russia-review.pdf
b1-open-source-software-survey-2024.pdf
b1-survey-climate-risks-for-banking.pdf
b1-dsight-venture-eurasia-results-1h-2024-review.pdf
b1-expert-ra-corporate-learning-research-july-2024.pdf
b1-banking-trends-for-consumers-survey-july-2024.pdf
b1-asi-tourism-in-russia-survey-june-2024.pdf
b1-outsourcing-in-russia-perspectives-survey-june-2024.pdf
b1-asi-tourism-survey-june-2024.pdf
b1-consumer-products-sector-survey-may

invalid pdf header: b'RIFF\x8c'
EOF marker not found


Супертренд на здоровое питание — ключевая тенденция на рынке хлебобулочных и кондитерских изделий в России
Error processing PDF Супертренд на здоровое питание — ключевая тенденция на рынке хлебобулочных и кондитерских изделий в России: Stream has ended unexpectedly
Самозанятость: предпринимательство и стабильное партнерство с платформами
Цифровая трансформация транспорта и логистики: готовность российских компаний


In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyPDFLoader
from langchain.schema import Document
import tempfile


llm = ChatOpenAI(model_name="gpt-4o",)

for doc in docs:
    name = doc['name']
    tags = doc['tags']
    additional_text = doc['text']
    
    for pdf in doc['pdfs']:
        pdf_name = pdf['name']
        print(pdf_name)
        pdf_content = pdf['content']
        try:
            with tempfile.NamedTemporaryFile() as temp_file:
                temp_file.write(pdf_content)
                temp_file_path = temp_file.name
                loader = PyPDFLoader(temp_file_path)
                pdf_data = loader.load()
        except :
            pdf_data = None
        if pdf_data:
            full_text = "\n".join([doc.page_content for doc in pdf_data])

            prompt = f"""
            **Название документа:**  {name}
            **Теги документа:** {", ".join(tags)} 
            **Краткое содержание документа с сайта (возможны html артефакты):**  {additional_text}
            **Документ:** {full_text}

            {f"""
            Перед тобой текст, полученный из PDF-документа. Название документа: {pdf_name}. Он может содержать пропуски, так как изображения и таблицы не были извлечены.

            **Твоя задача** – провести детальный отраслевой анализ на основе этого текста на русском языке. Если информация обрывочна, попытайся восстановить логический контекст или отметь, какие данные отсутствуют.

            **Формат анализа:**
            1. **Ключевые выводы** – выдели главных мыслей.
            2. **Тенденции в отрасли** – связаны ли идеи с актуальными событиями или трендами? краткосрочные и долгосрочные прогнозы.
            3. **Анализ рисков, неопределённости и возможностей** – есть ли критические моменты или перспективные направления?
            4. **Основные показатели** – ключевые цифры, тренды, прогнозы.
            5. **Рыночная динамика** – влияние факторов на отрасль.

            **Выведи аналитический отчёт в структурированном виде:**
            """}
            """

            # summarize_chain = load_summarize_chain(llm, chain_type="stuff")
            # summary = summarize_chain.invoke([Document(page_content=prompt)])
            pdf['content'] = prompt
            pdf['summary'] = 'a'# summary['output_text']
        else:
            pdf['content'] = ''
            pdf['summary'] = ''

In [7]:
docs[0]

{'link': '/analytics/b1-dsight-venture-eurasia-2024-results-review/',
 'date': datetime.datetime(2025, 2, 21, 0, 0),
 'name': 'Венчурная Евразия: итоги 2024 года',
 'name_en': 'b1-dsight-venture-eurasia-2024-results-review',
 'tags': ['ТМТ', 'Услуги в области подготовки IPO', 'Финансовый сектор'],
 'pdf_links': ['https://b1.ru//upload/sprint.editor/35d/zrkaela58rjysiwgvku7mu8ezg4wabv3/b1-dsight-venture-eurasia-2024-results-review.pdf',
  'https://b1.ru//local/assets/surveys/b1-dsight-venture-eurasia-2024-results-review.pdf',
  'https://b1.ru/local/assets/surveys/b1-dsight-venture-eurasia-2024-results-review.pdf'],
 'text': 'ТМТУслуги в области подготовки IPOФинансовый сектор\n\nСкачать\n\nАналитическая компания Dsight совместно с Группой компаний Б1, Московским венчурным фондом, SberUnity и ORT.Ventures провели исследование, посвященное объему инвестиций и динамике основных показателей венчурного рынка России и мира в 2024 году. \n\nСогласно результатам исследования, 2024 год стал пери

In [5]:
with open('temp.pdf', 'wb') as temp_file:
    temp_file.write(pdf_content)

In [6]:
doc['link']

'https://strategy.ru/research/research/supertrend-na-zdorovoe-pitanie-klyuchevaya-tendentsiya-na-rynke-khlebobulochnykh-i-konditerskikh-izdeliy-v-rossii-32/'

In [ ]:
from langchain.chains.summarize import load_summarize_chain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFLoader
from langchain.schema import Document


# Собираем текст в один блок
full_text = "\n".join([doc.page_content for doc in documents])

prompt = f"""
**Название документа:**  {name}
**Теги документа:** {", ".join(tags)} 
**Краткое содержание документа с сайта (возможны html артефакты):**  {additional_text}
**Документ:** {full_text}

{f"""
Перед тобой текст, полученный из PDF-документа. Название документа: {pdf_name}. Он может содержать пропуски, так как изображения и таблицы не были извлечены.

**Твоя задача** – провести детальный отраслевой анализ на основе этого текста на русском языке. Если информация обрывочна, попытайся восстановить логический контекст или отметь, какие данные отсутствуют.

**Формат анализа:**
1. **Ключевые выводы** – выдели главных мыслей.
2. **Тенденции в отрасли** – связаны ли идеи с актуальными событиями или трендами? краткосрочные и долгосрочные прогнозы.
3. **Анализ рисков, неопределённости и возможностей** – есть ли критические моменты или перспективные направления?
4. **Основные показатели** – ключевые цифры, тренды, прогнозы.
5. **Рыночная динамика** – влияние факторов на отрасль.

**Выведи аналитический отчёт в структурированном виде:**
"""}
"""

# Запускаем саммаризацию
summarize_chain = load_summarize_chain(llm, chain_type="stuff")
summary = summarize_chain.invoke([Document(page_content=prompt)])

print(summary['output_text'])

**Ключевые выводы:**
1. Темпы роста российской экономики замедляются из-за высоких процентных ставок, однако спрос на товары и услуги превышает предложение, что ведет к росту цен на промышленную продукцию.
2. Промышленное производство и деревообработка демонстрируют уверенный рост, опережая другие отрасли экономики.
3. Компании лесопромышленного комплекса (ЛПК) сталкиваются с трудностями в международных платежах и полагаются больше на собственные ресурсы, чем на государственную поддержку.
4. ЛПК надеется на ослабление санкций, что может стать значимым фактором для развития сектора.
5. Налоговая реформа 2025 года может усилить фискальную политику и повысить эффективность налогового администрирования.

**Тенденции в отрасли:**
- Активный рост деревообрабатывающей промышленности, наряду с машиностроением и металлургией в 2024 году.
- Рост в деревообработке обусловлен увеличением активности в полиграфической отрасли.
- Химическая промышленность увеличивает выпуск благодаря импортозамещению

In [146]:
print(summary)

In October and November 2024, the B1 Group, in collaboration with the Association of Paper Industry Specialists (ASBO), conducted an annual survey of the Russian forest industry, involving around 30 companies. The study revealed that the Russian economy's growth is slowing due to high interest rates, with demand for goods and services exceeding supply, leading to rising industrial product prices. The forest processing industry showed recovery, ranking among the top three growing sectors alongside engineering and metallurgy. Companies face challenges with international payments and are becoming more self-reliant, given limited government support and the hope for sanction relief. The upcoming 2025 tax reform is viewed both as a fiscal policy enhancement and a tool for efficient tax administration.

Key drivers for industry growth include increased activity in the printing sector for woodworking and demand from machinery and construction sectors for metallurgy. Companies are adapting by e

In [127]:
summary['output_text']

'В декабре 2024 года было проведено исследование состояния лесопромышленного комплекса России, которое выявило ключевые тренды и вызовы в отрасли. Несмотря на экономические и экологические трудности, сектор демонстрирует устойчивость и потенциал для роста, особенно благодаря внедрению инновационных технологий и повышению эффективности производства. Основное внимание уделяется устойчивому управлению лесными ресурсами и ответственному лесопользованию. Государственная поддержка и инвестиционная привлекательность остаются важными для модернизации инфраструктуры и привлечения новых технологий.\n\nПерспективы экспорта в свете изменений глобальных цепочек поставок и спроса на экологически чистые материалы требуют адаптации к международным стандартам. Российские компании в лесопромышленности сталкиваются с трудностями в международных платежах и снижением возможностей кредитования, что требует адаптации и поиска новых решений.\n\nОтмечается, что российская экономика в 2024 году замедляется, но 

In [16]:
with open('analytics.pkl', 'rb') as f:
    analytics = pickle.load(f)

In [20]:
analytics[0].keys()

dict_keys(['link', 'date', 'name', 'name_en', 'tags', 'text', 'pdf_links', 'pdfs'])

In [24]:
import PyPDF2
import pdfplumber
import tempfile


def extract_table(pdf_path, page_num, table_num):
    pdf = pdfplumber.open(pdf_path)
    table_page = pdf.pages[page_num]
    table = table_page.extract_tables()[table_num]
    return table


def table_converter(table):
    table_string = ''
    for row_num in range(len(table)):
        row = table[row_num]
        # Удаляем разрыв строки из текста с переносом
        cleaned_row = [item.replace('\n', ' ') if item is not None and '\n' in item else 'None' if item is None else item for item in row]
        # Преобразуем таблицу в строку
        table_string+=('|'+'|'.join(cleaned_row)+'|'+'\n')
    # Удаляем последний разрыв строки
    table_string = table_string[:-1]
    return table_string

In [27]:
with tempfile.NamedTemporaryFile() as temp_file:
    temp_file.write(analytics[0]['pdfs'][0]['content'])
    temp_file_path = temp_file.name
    pdfReaded = PyPDF2.PdfReader(temp_file_path)

In [ ]:
pdf_path = 'OFFER 3.pdf'

# создаём объект файла PDF
pdfFileObj = open(pdf_path, 'rb')
# создаём объект считывателя PDF
pdfReaded = PyPDF2.PdfReader(pdfFileObj)

# Создаём словарь для извлечения текста из каждого изображения
text_per_page = {}
# Извлекаем страницы из PDF
for pagenum, page in enumerate(extract_pages(pdf_path)):
    
    # Инициализируем переменные, необходимые для извлечения текста со страницы
    pageObj = pdfReaded.pages[pagenum]
    page_text = []
    line_format = []
    text_from_images = []
    text_from_tables = []
    page_content = []
    # Инициализируем количество исследованных таблиц
    table_num = 0
    first_element= True
    table_extraction_flag= False
    # Открываем файл pdf
    pdf = pdfplumber.open(pdf_path)
    # Находим исследуемую страницу
    page_tables = pdf.pages[pagenum]
    # Находим количество таблиц на странице
    tables = page_tables.find_tables()


    # Находим все элементы
    page_elements = [(element.y1, element) for element in page._objs]
    # Сортируем все элементы по порядку нахождения на странице
    page_elements.sort(key=lambda a: a[0], reverse=True)

    # Находим элементы, составляющие страницу
    for i,component in enumerate(page_elements):
        # Извлекаем положение верхнего края элемента в PDF
        pos= component[0]
        # Извлекаем элемент структуры страницы
        element = component[1]
        
        # Проверяем, является ли элемент текстовым
        if isinstance(element, LTTextContainer):
            # Проверяем, находится ли текст в таблице
            if table_extraction_flag == False:
                # Используем функцию извлечения текста и формата для каждого текстового элемента
                (line_text, format_per_line) = text_extraction(element)
                # Добавляем текст каждой строки к тексту страницы
                page_text.append(line_text)
                # Добавляем формат каждой строки, содержащей текст
                line_format.append(format_per_line)
                page_content.append(line_text)
            else:
                # Пропускаем текст, находящийся в таблице
                pass

        # Проверяем элементы на наличие изображений
        if isinstance(element, LTFigure):
            # Вырезаем изображение из PDF
            crop_image(element, pageObj)
            # Преобразуем обрезанный pdf в изображение
            convert_to_images('cropped_image.pdf')
            # Извлекаем текст из изображения
            image_text = image_to_text('PDF_image.png')
            text_from_images.append(image_text)
            page_content.append(image_text)
            # Добавляем условное обозначение в списки текста и формата
            page_text.append('image')
            line_format.append('image')

        # Проверяем элементы на наличие таблиц
        if isinstance(element, LTRect):
            # Если первый прямоугольный элемент
            if first_element == True and (table_num+1) <= len(tables):
                # Находим ограничивающий прямоугольник таблицы
                lower_side = page.bbox[3] - tables[table_num].bbox[3]
                upper_side = element.y1 
                # Извлекаем информацию из таблицы
                table = extract_table(pdf_path, pagenum, table_num)
                # Преобразуем информацию таблицы в формат структурированной строки
                table_string = table_converter(table)
                # Добавляем строку таблицы в список
                text_from_tables.append(table_string)
                page_content.append(table_string)
                # Устанавливаем флаг True, чтобы избежать повторения содержимого
                table_extraction_flag = True
                # Преобразуем в другой элемент
                first_element = False
                # Добавляем условное обозначение в списки текста и формата
                page_text.append('table')
                line_format.append('table')

            # Проверяем, извлекли ли мы уже таблицы из этой страницы
            if element.y0 >= lower_side and element.y1 <= upper_side:
                pass
            elif not isinstance(page_elements[i+1][1], LTRect):
                table_extraction_flag = False
                first_element = True
                table_num+=1


    # Создаём ключ для словаря
    dctkey = 'Page_'+str(pagenum)
    # Добавляем список списков как значение ключа страницы
    text_per_page[dctkey]= [page_text, line_format, text_from_images,text_from_tables, page_content]

# Закрываем объект файла pdf
pdfFileObj.close()

# Удаляем созданные дополнительные файлы
os.remove('cropped_image.pdf')
os.remove('PDF_image.png')

# Удаляем содержимое страницы
result = ''.join(text_per_page['Page_0'][4])
print(result)